企业根据自身业务需求和数据特点，定制化开发或优化大型人工智能模型形成企业私有大模型，从技术层面来讲，实现企业私有大模型有2个技术手段，微调（Fine-tuning）和RAG（Retrieval Augmented Generation）检索增强生成，更大的模型带来了更好的性能，但是以传统微调对模型进行全量参数微调会耗费极其昂贵的算力资源，关键参数规模上的日益增大也会加剧对显存的依赖。

参数高效微调通常是指在机器学习和深度学习中，对预训练模型的参数进行少量调整以适应新任务的过程，这个过程可以提高模型的泛化能力，同时减少训练时间和资源消耗，参数高效微调方法仅对模型的一小部分参数（这一小部分可能是模型自身的，也可能是外部引引入的）进行训练，便可以为模型带来显著的性能变化，《Scaling Down to Scale Up A Guide to Parameter-Efficient Fine-Tuning》论文里列出了常用的参数高效微调的方式。本文是基于LoRA进行高效微调。

![](https://opendatascience.com/wp-content/uploads/2023/10/PEFT-640x300.png)

LoRA（Low-Rank Adaptation）是一种技术，通过低秩分解将权重更新表示为两个较小的矩阵（称为更新矩阵），从而加速大型模型的微调，并减少内存消耗，为了使微调更加高效，LoRA的方法是通过低秩分解，使用两个较小的矩阵（称为更新矩阵）来表示权重更新，这些新矩阵可以通过训练适应新数据，同时保持整体变化的数量较少，原始的权重矩阵保持冻结，不再接收任何进一步的调整，为了产生最终结果，同时使用原始和适应后的权重进行合并。

#### 模型推理测试

In [3]:
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,Trainer,TrainingArguments
from datasets import load_dataset
from peft import LoraConfig,TaskType,get_peft_model
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("Qwen2-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen2-0.5B-Instruct",low_cpu_mem_usage=True)

model = model.cuda()

ipt = tokenizer("Human: {}".format("谁是周杰伦？") + "\n\nAssistant: ", return_tensors="pt").to(model.device)
re = tokenizer.decode(model.generate(**ipt,max_length=256,do_sample=False)[0],skip_special_tokens=True)
print(re)

e:\miniconda3\envs\llamafac\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AssertionError: Torch not compiled with CUDA enabled

#### 数据下载

为了使用peft的lora微调我们的0.5B模型，我们需要下载数据集。数据集下载地址为：https://huggingface.co/datasets/shibing624/alpaca-zh

#### Qwen微调

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,Trainer,TrainingArguments
from datasets import load_dataset
from peft import LoraConfig,TaskType,get_peft_model
import torch

# 加载数据
dataset = load_dataset('json',data_files='data/alpaca_gpt4_data_zh.json')
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 48818
    })
})

> 注意：非常关键一点，要区分Dataset和DatasetDict的关系，DatasetDict包含了Dataset，部分下载的数据集包括train和test两个Dataset，因此后续再进行数据集划分的时候，必须使用ds[“train”]再调用train_test_split方法，因为DatasetDict没有这个方法。

有时候需要查看数据集内相关信息，为后续数据的处理做进一步的判断和分析：

In [2]:
dataset["train"].column_names

['instruction', 'input', 'output']

进一步查看数据集的某一个字段：

In [3]:
dataset["train"]["output"][:1]

['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。']

每个类型的具体属性：

In [4]:
dataset["train"].features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None)}

按照比例进行划分，例如按照训练集为90%，测试集为10%进行划分：

In [5]:
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 43936
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 4882
    })
})

在整个环节，数据集的处理是非常重要的过程。总的来说dataset是一条数据，而dataloader是一个批次数据，针对每一条数据使用数据集映射函数(即下方process_fuc函数进行处理)。

In [6]:
# 数据集映射

tokenizer = AutoTokenizer.from_pretrained("Qwen2-0.5B-Instruct")

def process_fuc(one):
    MAX_LENGTH = 256
    input_ids,attention_mask,labels = [],[],[]
    instruction = tokenizer("\n".join(["Human: "+ one["instruction"],one["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(one["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


使用map进行映射处理：

In [7]:
tokenizer_dataset = dataset.map(process_fuc,remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/43936 [00:00<?, ? examples/s]

Map:   0%|          | 0/4882 [00:00<?, ? examples/s]

In [8]:
tokenizer_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 43936
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4882
    })
})

可以看出它是一个字典类型，每个里面包括了经过tokenizer后的'input_ids', 'attention_mask', 'labes'三个字段。

我们可以通过tokenizer.decode进行反向解码验证:

In [9]:
tokenizer.decode(tokenizer_dataset["train"][0]["input_ids"])

'Human: 复杂碳水化合物的例子是什么？\n\nAssistant: 复杂碳水化合物，也叫多糖（polysaccharide），是由许多单糖分子通过缩合反应连接在一起而形成的长链状大分子。 一些常见的复杂碳水化合物的例子包括：\n\n- 纤维素： 纤维素是一种由葡萄糖单元组成的线性多糖，它是植物细胞壁的主要成分。纤维素不能被人类消化，但是能够通过帮助食物在肠道中移动来促进肠道健康。\n- 淀粉： 淀粉是由多个葡萄糖分子组成的多糖，是植物储存能量的主要方式。大多数淀粉分子具有分支结构，它可以在消化道内迅速分解为葡萄糖，为人体提供能量。淀粉广泛存在于谷物，蔬菜和豆类等食物中。\n- 糖原： 糖原是由许多葡萄糖单元组成的多糖，是动物储存能量的主要方式。它主要储存在肝脏和肌肉中，在需要能量的时候迅速分解为葡萄糖，为人体提供能量。<|im_end|>'

可以看出上述正是我们拼接完成的内容。

加载Qwen2模型，这里默认是单精度：

In [10]:
model = AutoModelForCausalLM.from_pretrained("Qwen2-0.5B-Instruct",low_cpu_mem_usage=True)

In [11]:
model.dtype

torch.float32

我们可以通过torch_dtype参数进行调整，设置为半精度，可以显著减少内存占用：

In [12]:
model = AutoModelForCausalLM.from_pretrained("Qwen2-0.5B-Instruct", low_cpu_mem_usage=True, torch_dtype=torch.half)

In [13]:
model.dtype

torch.float16

配置lora微调参数，这里还有一个参数lora_alpha用于设置lora效果的整体权重设置，这里默认即可。另外一个很重要的就是lora到底微调了哪些参数，实际上具体是由target_modules参数决定的。

In [14]:
loraconfig = LoraConfig(task_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, loraconfig)

In [17]:
# 配置训练参数
args = TrainingArguments(
    output_dir="./chatbot", # output_dir参数通常用于指定训练过程中生成的输出文件（如模型权重、日志文件、检查点等）的存储目录
    per_device_train_batch_size=1, # 训练时的batch_size
    logging_steps=10, # log 打印的频率
    num_train_epochs=1 # 训练轮数
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenizer_dataset['train'],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
)

In [18]:
# 开始训练
trainer.train()

  0%|          | 0/43936 [00:00<?, ?it/s]

{'loss': 2.2837, 'grad_norm': 1.6679210662841797, 'learning_rate': 4.9988619810633654e-05, 'epoch': 0.0}


OutOfMemoryError: CUDA out of memory. Tried to allocate 132.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 4.90 GiB is allocated by PyTorch, and 252.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

使用指令微调后模型进行推理结果,LoRA微调的训练中，会持续在指定的output_dir="./chatbot"位置保存模型权重,即训练过程会持续在当前的chatbot文件夹下创建checkpoint-子文件夹，每个checkpoint子文件夹下会存放LoRA微调后的模型。

那么我们如何调用未调后的模型呢？

分两步走：首先是加载基础模型，然后是使用PeftModel的from_pretrained的方法进一步加载，包括两个参数，第一个参数就是基础模型，第二个参数model_id就是指定前述的checkpoint文件夹。

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,Trainer,TrainingArguments
from datasets import load_dataset
from peft import LoraConfig,TaskType,get_peft_model
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("Qwen2-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen2-1.5B-Instruct",low_cpu_mem_usage=True)

model = model.cuda()
lora_model = PeftModel.from_pretrained(model, model_id="./chatbot/checkpoint-43937/")

ipt = tokenizer("Human: {}\n{}".format("如何关闭华硕主板的xmp功能？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
re = tokenizer.decode(lora_model.generate(**ipt,max_length=256,do_sample=False)[0],skip_special_tokens=True)
print(re)

微调后模型合并。微调后我们可以将LoRA训练部分的参数和原基础模型进行合并，这样我们后续可以直接调用这个合并后的模型：

In [ ]:
mergemodel = lora_model.merge_and_unload()
mergemodel.save_pretrained("./merge_model")

#